In [1]:
import os
import pandas as pd
import re

In [2]:
INPUT_PATH = r"E:\数据集\ShipEar\data_preprocessing\annotation\shipsEar.xlsx"
OUT_PATH = r"E:\数据集\ShipEar\data_preprocessing\annotation"

df = pd.read_excel(INPUT_PATH)
csv_output_file = os.path.join(OUT_PATH, "shipsEar.csv")
df.to_csv(csv_output_file, index=False, encoding='utf-8-sig', errors='ignore')

In [4]:
df = pd.read_csv(csv_output_file)
out_df = pd.DataFrame()
for col in ["ID", "Filename", "Name", "Type", "Channel Depth", "Wind", "Distance"]:
    out_df[col] = df[col]
out_df['group_id'] = df['ID']
output_file = os.path.join(OUT_PATH, "shipear_group.csv")
out_df.to_csv(output_file, index=False)
print("输入文件:", INPUT_PATH)
print("已保存:", output_file)

输入文件: E:\数据集\ShipEar\data_preprocessing\annotation\shipsEar.xlsx
已保存: E:\数据集\ShipEar\data_preprocessing\annotation\shipear_group.csv


In [5]:
type_mapping = {
    'Fishboat': 0,
    'Trawler': 0,
    'Mussel boat': 0,
    'Tugboat': 0,
    'Dredger': 0,

    'Motorboat': 1,
    'Pilot ship': 1,
    'Sailboat': 1,

    'Passengers': 2,

    'Ocean liner': 3,
    'RORO': 3,

    'Natural ambient noise': 4
}
out_df['class_id'] = out_df['Type'].map(type_mapping)
output_file = os.path.join(OUT_PATH, "shipear_group_class.csv")
out_df.to_csv(output_file, index=False)

In [6]:
df = pd.read_csv(output_file)
out_df = pd.DataFrame()
for col in ["ID", "Filename", "Name", "Type", "group_id", "class_id"]:
    out_df[col] = df[col]
    
file_directory = r"E:\数据集\ShipEar\data_preprocessing\4_Frame_Windows_2s_50%"
if os.path.exists(file_directory):
    files_in_dir = [os.path.splitext(f)[0] for f in os.listdir(file_directory) if os.path.isfile(os.path.join(file_directory, f))]
    out_df['segmented_filename'] = out_df['Filename'].str.replace('.wav', '', regex=False)

    # 创建原始文件名到信息的映射
    original_file_mapping = {}
    for _, row in out_df.iterrows():
        original_filename = row['Filename']
        original_file_mapping[original_filename] = row.to_dict()
    # 为每个分段文件创建标注记录
    segmented_rows = []
    for segmented_file in files_in_dir:
        parts = segmented_file.split('_')
        original_filename = '_'.join(parts[:-1]) + '.wav'
        segment_number = parts[-1]

        if original_filename in original_file_mapping:
            original_info = original_file_mapping[original_filename]
            segmented_row = original_info.copy()
            segmented_row['segmented_filename'] = segmented_file
            segmented_row['segment_number'] = segment_number
            segmented_rows.append(segmented_row)

    if segmented_rows:
        segmented_df = pd.DataFrame(segmented_rows)
        column_order = ['ID', 'Filename', 'Name', 'Type', 'group_id', 'class_id', 'segmented_filename', 'segment_number']
        segmented_df = segmented_df[column_order]
        output_file = os.path.join(OUT_PATH, "shipear_group_class_segmented.csv")
        segmented_df.to_csv(output_file, index=False)
        print(f"已处理 {len(segmented_rows)} 个分段文件")
        print(f"已保存: {output_file}")

已处理 11210 个分段文件
已保存: E:\数据集\ShipEar\data_preprocessing\annotation\shipear_group_class_segmented.csv
